In [2]:
import aiohttp
import asyncio
from aiohttp import ClientSession
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()
api_key = os.getenv('API_KEY')
api_host = os.getenv('API_HOST')
competetions = ['L1', 'GB1', 'ES1', 'IT1', 'FR1']
seasons = list(range(2000, 2021))
next_delay = 0.3


In [4]:
teams_per_season = {}

for league in competetions:
    teams_per_season[league] = {}
    for season in seasons:
        teams_per_season[league][season] = {}

In [5]:
async def get_table(league, season, session):
    global next_delay
    
    next_delay += 0.3
    await asyncio.sleep(next_delay)
    
    url = 'https://transfermarket.p.rapidapi.com/competitions/get-table'
    querystring = {'id':league, 'seasonID':season}
    headers = {
        'X-RapidAPI-Key': api_key,
        'X-RapidAPI-Host': api_host
        }
    
    async with session.get(url, headers=headers, params=querystring) as response:
        json = await response.json()
        if json.get('table') is not None:
            table = json.get('table')
            teams_per_season[league][season] = [int(table[i].get('id')) for i in range(len(table))]
            return True
        return False
        

async def get_tables(competetions, seasons):
    
    async with aiohttp.ClientSession() as session:
        tasks = []
        for league in competetions:
            for season in seasons:  
                if not teams_per_season[league][season]:
                    tasks.append(asyncio.ensure_future(get_table(league, season, session)))
        await asyncio.gather(*tasks)
        
    global next_delay
    next_delay = 0.3


In [6]:
await get_tables(competetions, seasons)

In [7]:
# write teams_per_season to csv
import csv
with open('teams_per_season.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=';')
    for league in competetions:
        for season in seasons:
            if teams_per_season[league][season]:
                writer.writerow([league, season, teams_per_season[league][season]])